In [40]:
import json
from typing import Dict, List, Optional

import json
from typing import Dict, List, Set, Any, Optional


import json
from typing import Dict, List, Set, Any, Optional
import re

class JSONValidator:
    def __init__(self):
        self.errors = []
        self.warnings = []
        self.scope_symbols = {}  # {scope_level: {var_name: var_info}}
        self.all_scopes = []  # Сохраняем все scopes для поиска родительских
        self.functions = {}  # {func_name: func_info}
        self.source_map = {}  # Сопоставление узлов с исходными строками
        self.builtin_functions = {
            "print": {"return_type": "None", "min_args": 0, "max_args": None},
            "len": {"return_type": "int", "min_args": 1, "max_args": 1},
            "str": {"return_type": "string", "min_args": 1, "max_args": 1},
            "int": {"return_type": "int", "min_args": 1, "max_args": 1},
            "bool": {"return_type": "bool", "min_args": 1, "max_args": 1},
            "range": {"return_type": "range", "min_args": 1, "max_args": 3}
        }
        # Для отслеживания состояния переменных
        self.variable_history = {}  # {(scope_level, var_name): [{"action": "declare"/"assign"/"delete", "node_id": str}]}
        self.variable_states = {}  # {(scope_level, var_name): "active"/"deleted"}
    
    def validate(self, json_data: List[Dict]) -> Dict:
        """Основной метод валидации"""
        self.errors = []
        self.warnings = []
        self.scope_symbols = {}
        self.functions = {}
        self.all_scopes = json_data
        self.source_map = {}
        self.variable_history = {}
        self.variable_states = {}
        
        # Собираем информацию о всех узлах и их строках
        self.build_source_map(json_data)
        
        if not isinstance(json_data, list):
            self.add_error("JSON должен быть списком scope'ов")
            return self.get_report()
        
        # Собираем информацию о всех scope'ах и символах
        self.collect_symbols(json_data)
        
        # Строим историю переменных
        self.build_variable_history(json_data)
        
        # Проверяем каждый scope
        for scope_idx, scope in enumerate(json_data):
            self.validate_scope(scope, scope_idx, json_data)
        
        return self.get_report()
    
    def collect_symbols(self, json_data: List[Dict]):
        """Собирает информацию о всех символах в системе"""
        for scope_idx, scope in enumerate(json_data):
            level = scope.get("level", 0)

            if "symbol_table" in scope and scope["symbol_table"]:
                
                if level not in self.scope_symbols:
                    self.scope_symbols[level] = {}
                
                for symbol_name, symbol_info in scope["symbol_table"].items():
                    # Сохраняем функции отдельно
                    if symbol_info.get("key") == "function":
                        self.functions[symbol_name] = symbol_info
                    else:
                        # Сохраняем обычные переменные
                        self.scope_symbols[level][symbol_name] = symbol_info
    
    def build_source_map(self, json_data: List[Dict]):
        """Строит карту соответствия узлов исходным строкам"""
        for scope_idx, scope in enumerate(json_data):
            level = scope.get("level", 0)
            graph = scope.get("graph", [])
            
            for node_idx, node in enumerate(graph):
                node_id = f"{scope_idx}.{node_idx}"
                # Сохраняем содержимое строки из поля 'content'
                self.source_map[node_id] = {
                    "content": node.get("content", ""),
                    "scope_idx": scope_idx,
                    "scope_level": level,
                    "scope_type": scope.get("type", "unknown")
                }
    
    def build_variable_history(self, json_data: List[Dict]):
        """Строит историю операций с переменными С УЧЕТОМ ПОРЯДКА СТРОК"""
        # Сначала собираем все узлы в правильном порядке
        all_nodes = []
        
        for scope_idx, scope in enumerate(json_data):
            level = scope.get("level", 0)
            graph = scope.get("graph", [])
            
            for node_idx, node in enumerate(graph):
                node_id = f"{scope_idx}.{node_idx}"
                all_nodes.append({
                    "scope_idx": scope_idx,
                    "node_idx": node_idx,
                    "node_id": node_id,
                    "level": level,
                    "node": node,
                    "scope": scope
                })
        
        # Сортируем по scope_idx и node_idx для сохранения порядка
        all_nodes.sort(key=lambda x: (x["scope_idx"], x["node_idx"]))
        
        # Глобальный счетчик времени для всех операций
        global_timestamp = 0
        
        for node_info in all_nodes:
            scope_idx = node_info["scope_idx"]
            node_idx = node_info["node_idx"]
            node_id = node_info["node_id"]
            level = node_info["level"]
            node = node_info["node"]
            node_type = node.get("node", "unknown")
            
            if node_type == "declaration":
                symbols = node.get("symbols", [])
                for symbol in symbols:
                    key = (level, symbol)
                    if key not in self.variable_history:
                        self.variable_history[key] = []
                    self.variable_history[key].append({
                        "action": "declare",
                        "node_id": node_id,
                        "content": node.get("content", ""),
                        "timestamp": global_timestamp,
                        "unique_id": f"{node_id}_{symbol}"
                    })
                    global_timestamp += 1
                    # При объявлении переменная активна
                    self.variable_states[key] = "active"
            
            elif node_type == "assignment":
                symbols = node.get("symbols", [])
                for symbol in symbols:
                    key = (level, symbol)
                    if key not in self.variable_history:
                        self.variable_history[key] = []
                    self.variable_history[key].append({
                        "action": "assign",
                        "node_id": node_id,
                        "content": node.get("content", ""),
                        "timestamp": global_timestamp,
                        "unique_id": f"{node_id}_{symbol}"
                    })
                    global_timestamp += 1
            
            elif node_type == "delete":
                symbols = node.get("symbols", [])
                for symbol in symbols:
                    key = (level, symbol)
                    if key not in self.variable_history:
                        self.variable_history[key] = []
                    self.variable_history[key].append({
                        "action": "delete",
                        "node_id": node_id,
                        "content": node.get("content", ""),
                        "timestamp": global_timestamp,
                        "unique_id": f"{node_id}_{symbol}"
                    })
                    global_timestamp += 1
                    # Переменная помечается как удаленная
                    self.variable_states[key] = "deleted"
            
            elif node_type == "builtin_function_call":
                func_name = node.get("function", "")
                args = node.get("arguments", [])
                dependencies = node.get("dependencies", [])
                
                # Используем dependencies из узла
                for dep in dependencies:
                    if dep and dep.isalpha() and dep not in ["True", "False", "None"]:
                        key = (level, dep)
                        if key not in self.variable_history:
                            self.variable_history[key] = []
                        self.variable_history[key].append({
                            "action": "use_in_function",
                            "function": func_name,
                            "node_id": node_id,
                            "content": node.get("content", ""),
                            "timestamp": global_timestamp,
                            "unique_id": f"{node_id}_{dep}"
                        })
                        global_timestamp += 1
            
            elif node_type == "function_call":
                args = node.get("arguments", [])
                dependencies = node.get("dependencies", [])
                for dep in dependencies:
                    if dep and dep.isalpha() and dep not in ["True", "False", "None"]:
                        key = (level, dep)
                        if key not in self.variable_history:
                            self.variable_history[key] = []
                        self.variable_history[key].append({
                            "action": "use_in_function",
                            "node_id": node_id,
                            "content": node.get("content", ""),
                            "timestamp": global_timestamp,
                            "unique_id": f"{node_id}_{dep}"
                        })
                        global_timestamp += 1
    
    def get_variable_state(self, var_name: str, level: int) -> str:
        """Получает текущее состояние переменной"""
        key = (level, var_name)
        return self.variable_states.get(key, "unknown")
    
    def is_variable_deleted(self, var_name: str, level: int) -> bool:
        """Проверяет, удалена ли переменная"""
        state = self.get_variable_state(var_name, level)
        return state == "deleted"
    
    def get_last_variable_action(self, var_name: str, level: int) -> Optional[Dict]:
        """Получает последнее действие с переменной"""
        key = (level, var_name)
        if key in self.variable_history and self.variable_history[key]:
            return self.variable_history[key][-1]
        return None
    
    def add_error(self, message: str, scope_idx: int = None, node_idx: int = None):
        """Добавляет ошибку с информацией о строке"""
        full_message = message
        
        if scope_idx is not None and node_idx is not None:
            node_id = f"{scope_idx}.{node_idx}"
            if node_id in self.source_map:
                content = self.source_map[node_id]["content"]
                if content:
                    full_message = f"Строка '{content}': {message}"
        
        self.errors.append(full_message)
    
    def add_warning(self, message: str, scope_idx: int = None, node_idx: int = None):
        """Добавляет предупреждение с информацией о строке"""
        full_message = message
        
        if scope_idx is not None and node_idx is not None:
            node_id = f"{scope_idx}.{node_idx}"
            if node_id in self.source_map:
                content = self.source_map[node_id]["content"]
                if content:
                    full_message = f"Строка '{content}': {message}"
        
        self.warnings.append(full_message)
    
    def validate_scope(self, scope: Dict, scope_idx: int, all_scopes: List[Dict]):
        """Валидирует отдельный scope"""
        level = scope.get("level", 0)
        scope_type = scope.get("type", "unknown")
        
        required_fields = ["level", "type", "local_variables", "graph", "symbol_table"]
        for field in required_fields:
            if field not in scope:
                self.add_error(f"Scope {scope_idx} (level {level}, type {scope_type}) отсутствует поле '{field}'", 
                             scope_idx, None)
        
        self.validate_symbol_table(scope, scope_idx)
        
        if "graph" in scope:
            self.validate_graph(scope, scope_idx)
        
        if scope_type == "function":
            self.validate_function_return(scope, scope_idx)
        
        self.validate_loops(scope, scope_idx)
    
    def validate_symbol_table(self, scope: Dict, scope_idx: int):
        """Валидирует таблицу символов scope'а"""
        symbol_table = scope.get("symbol_table", {})
        local_vars = scope.get("local_variables", [])
        
        for var_name in local_vars:
            if var_name not in symbol_table:
                self.add_warning(f"переменная '{var_name}' в local_variables отсутствует в symbol_table", 
                               scope_idx, None)
        
        for symbol_name, symbol_info in symbol_table.items():
            self.validate_symbol(symbol_info, scope_idx, symbol_name)
    
    def validate_symbol(self, symbol_info: Dict, scope_idx: int, symbol_name: str):
        """Валидирует отдельный символ"""
        required_fields = ["name", "key", "type", "id"]
        for field in required_fields:
            if field not in symbol_info:
                self.add_error(f"у символа '{symbol_name}' отсутствует поле '{field}'", 
                             scope_idx, None)
        
        if symbol_info.get("name") != symbol_name:
            self.add_warning(f"имя символа '{symbol_name}' не совпадает с полем 'name': {symbol_info.get('name')}", 
                           scope_idx, None)
        
        var_type = symbol_info.get("type", "")
        if var_type not in ["int", "string", "function", "None"] and not var_type.startswith("ptr_"):
            self.add_warning(f"символ '{symbol_name}' имеет неизвестный тип '{var_type}'", 
                           scope_idx, None)
        
        if symbol_info.get("key") == "const":
            if "value" not in symbol_info:
                self.add_error(f"константа '{symbol_name}' не имеет значения", 
                             scope_idx, None)
    
    def validate_graph(self, scope: Dict, scope_idx: int):
        """Валидирует граф операций"""
        graph = scope.get("graph", [])
        symbol_table = scope.get("symbol_table", {})
        level = scope.get("level", 0)
        
        for node_idx, node in enumerate(graph):
            node_type = node.get("node", "unknown")
            content = node.get("content", "")
            
            if node_type == "declaration":
                self.validate_declaration(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type == "assignment":
                print(f"    >>> Валидация assignment: {content}")
                self.validate_assignment(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type == "delete":
                print(f"    >>> Валидация delete: {content}")
                self.validate_delete(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type == "augmented_assignment":
                self.validate_augmented_assignment(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type == "function_declaration":
                self.validate_function_declaration(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type in ["function_call", "function_call_assignment"]:
                self.validate_function_call(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type in ["builtin_function_call", "builtin_function_call_assignment"]:
                self.validate_builtin_function_call(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type == "return":
                self.validate_return(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type in ["while_loop", "for_loop"]:
                self.validate_loop_node(node, node_idx, scope_idx, symbol_table, level)
            
            elif node_type == "print":
                print(f"    >>> Валидация print: {content}")
                self.validate_print(node, node_idx, scope_idx, symbol_table, level)
    
    def validate_declaration(self, node: Dict, node_idx: int, scope_idx: int, 
                       symbol_table: Dict, level: int):
        """Валидирует объявление переменной"""
        symbols = node.get("symbols", [])
        operations = node.get("operations", [])
        content = node.get("content", "")
        
        for symbol in symbols:
            if symbol not in symbol_table:
                self.add_error(f"объявляемая переменная '{symbol}' отсутствует в symbol_table", 
                            scope_idx, node_idx)
            else:
                for op in operations:
                    if op.get("type") in ["NEW_VAR", "NEW_CONST"]:
                        declared_type = op.get("var_type") or op.get("const_type")
                        actual_type = symbol_table[symbol].get("type")
                        if declared_type != actual_type:
                            self.add_error(f"тип переменной '{symbol}' не совпадает (объявлен: {declared_type}, в symbol_table: {actual_type})", 
                                        scope_idx, node_idx)
        
        # Проверяем значение инициализации
        if content:
            # Парсим объявление
            pattern = r"var\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*:\s*([a-zA-Z_][a-zA-Z0-9_]*)\s*=\s*(.+)"
            match = re.match(pattern, content)
            
            if match:
                var_name, var_type, value = match.groups()
                print(f"  Объявление переменной '{var_name}' с инициализацией '{value}'")
                
                # Проверяем выражение инициализации
                self.validate_expression(value, scope_idx, node_idx, level)
    
    def validate_expression(self, expression: str, scope_idx: int, node_idx: int, level: int):
        """Валидирует выражение (правая часть присваивания или инициализации)"""
        print(f"    Валидация выражения: '{expression}'")
        
        # 1. Проверяем вызовы функций
        func_calls = re.findall(r'\b([a-zA-Z_][a-zA-Z0-9_]*)\s*\(', expression)
        for func_name in func_calls:
            print(f"      Обнаружен вызов функции: {func_name}")
            
            # Проверяем, существует ли функция
            if func_name not in self.functions and func_name not in self.builtin_functions:
                self.add_error(f"функция '{func_name}' не объявлена", scope_idx, node_idx)
        
        # 2. Проверяем переменные
        # Регулярное выражение для поиска идентификаторов
        var_pattern = r'\b([a-zA-Z_][a-zA-Z0-9_]+)\b'
        identifiers = re.findall(var_pattern, expression)
        
        for identifier in identifiers:
            # Пропускаем ключевые слова, типы данных, литералы, функции
            if (identifier in ["True", "False", "None"] or
                identifier.isdigit() or
                (identifier.startswith('"') and identifier.endswith('"')) or
                (identifier.startswith("'") and identifier.endswith("'"))):
                continue
            
            # Проверяем, не является ли это вызовом функции (уже обработали)
            if identifier in func_calls:
                continue
            
            # Это переменная - проверяем ее существование
            print(f"      Проверка переменной '{identifier}' в выражении")
            if not self.find_symbol_in_scope(identifier, level):
                self.add_error(f"переменная '{identifier}' в выражении не объявлена", 
                            scope_idx, node_idx)
            elif self.is_variable_deleted(identifier, level):
                self.add_error(f"переменная '{identifier}' в выражении была удалена", 
                            scope_idx, node_idx)
    
    def validate_assignment(self, node: Dict, node_idx: int, scope_idx: int, 
                      symbol_table: Dict, level: int):
        """Валидирует присваивание"""
        symbols = node.get("symbols", [])
        dependencies = node.get("dependencies", [])
        operations = node.get("operations", [])
        content = node.get("content", "")
        
        # 1. Проверяем левую часть (целевую переменную)
        for symbol in symbols:
            symbol_info = self.get_symbol_info(symbol, level)
            print(f"  Проверка переменной '{symbol}': symbol_info={symbol_info is not None}")
            
            if not symbol_info:
                self.add_error(f"присваиваемая переменная '{symbol}' не объявлена", 
                            scope_idx, node_idx)
            else:
                # Проверяем, не была ли переменная удалена
                print(f"    Состояние переменной '{symbol}': {self.get_variable_state(symbol, level)}")
                print(f"    Удалена ли: {self.is_variable_deleted(symbol, level)}")
                
                if self.is_variable_deleted(symbol, level):
                    # Получаем историю переменной
                    key = (level, symbol)
                    if key in self.variable_history:
                        print(f"    История переменной {symbol}:")
                        for action in self.variable_history[key]:
                            print(f"      Действие: {action['action']}, content: {action.get('content', '')}")
                    
                    # Проверяем, была ли после удаления переинициализация
                    last_action = self.get_last_variable_action(symbol, level)
                    if last_action and last_action["action"] == "delete":
                        # Ищем объявления после удаления
                        found_redeclaration = False
                        for action in self.variable_history.get(key, []):
                            if action["action"] == "declare" and action["timestamp"] > last_action["timestamp"]:
                                found_redeclaration = True
                                break
                        
                        if not found_redeclaration:
                            self.add_error(f"переменная '{symbol}' была удалена и требует переинициализации", 
                                        scope_idx, node_idx)
                elif symbol_info.get("key") == "const":
                    self.add_error(f"попытка присваивания константе '{symbol}'", 
                                scope_idx, node_idx)
        
        # 2. Проверяем правую часть выражения
        # Извлекаем правую часть из content
        if symbols and content:
            target_var = symbols[0]
            # Вырезаем правую часть после "="
            if "=" in content:
                expression = content.split("=", 1)[1].strip()
                print(f"  Выражение в правой части: '{expression}'")
                
                # Проверяем вызовы функций в правой части
                self.validate_expression(expression, scope_idx, node_idx, level)
        
        # 3. Проверяем зависимости (используемые переменные)
        for dep in dependencies:
            print(f"  Проверка зависимости '{dep}'")
            found = self.find_symbol_in_scope(dep, level)
            print(f"    Найдена в scope: {found}")
            
            if not found:
                self.add_error(f"используемая переменная '{dep}' не объявлена", 
                            scope_idx, node_idx)
            elif self.is_variable_deleted(dep, level):
                print(f"    Переменная '{dep}' удалена: {self.is_variable_deleted(dep, level)}")
                self.add_error(f"используемая переменная '{dep}' была удалена", 
                            scope_idx, node_idx)
    
    def validate_delete(self, node: Dict, node_idx: int, scope_idx: int, 
                  symbol_table: Dict, level: int):
        """Валидирует удаление переменной"""
        symbols = node.get("symbols", [])
        node_id = f"{scope_idx}.{node_idx}"
        
        for symbol in symbols:
            symbol_info = self.get_symbol_info(symbol, level)
            if not symbol_info:
                self.add_error(f"удаляемая переменная '{symbol}' не объявлена", 
                            scope_idx, node_idx)
            else:
                if symbol_info.get("key") == "const":
                    self.add_error(f"попытка удаления константы '{symbol}'", 
                                scope_idx, node_idx)
                else:
                    # Проверяем историю переменной
                    key = (level, symbol)
                    if key in self.variable_history:
                        # Ищем уже существующие удаления
                        existing_deletes = [action for action in self.variable_history[key] 
                                        if action["action"] == "delete"]
                        
                        # Получаем timestamp текущего узла (если он уже добавлен в историю)
                        current_timestamp = None
                        for action in self.variable_history[key]:
                            if action.get("unique_id") == f"{node_id}_{symbol}":
                                current_timestamp = action.get("timestamp")
                                break
                        
                        if current_timestamp is not None:
                            # Проверяем, есть ли удаления до текущего момента
                            deletes_before_current = [d for d in existing_deletes 
                                                    if d["timestamp"] < current_timestamp]
                            
                            if deletes_before_current:
                                # Ищем, было ли объявление после последнего удаления
                                last_delete = max(deletes_before_current, key=lambda x: x["timestamp"])
                                declarations_after = [action for action in self.variable_history[key] 
                                                    if action["action"] == "declare" and 
                                                    action["timestamp"] > last_delete["timestamp"]]
                                
                                if not declarations_after:
                                    # Переменная уже была удалена и не переинициализирована
                                    self.add_error(f"переменная '{symbol}' уже была удалена", 
                                                scope_idx, node_idx)
    
    def validate_unary_operation(self, op: Dict, node_idx: int, scope_idx: int, level: int):
        """Валидирует унарную операцию"""
        value = op.get("value")
        
        if value and value.isalpha() and value not in ["True", "False", "None"]:
            if not self.find_symbol_in_scope(value, level):
                self.add_error(f"операнд унарной операции '{value}' не объявлен", 
                             scope_idx, node_idx)
            elif self.is_variable_deleted(value, level):
                self.add_error(f"операнд унарной операции '{value}' был удален", 
                             scope_idx, node_idx)
    
    def validate_augmented_assignment(self, node: Dict, node_idx: int, scope_idx: int, 
                                  symbol_table: Dict, level: int):
        """Валидирует составное присваивание"""
        symbols = node.get("symbols", [])
        operations = node.get("operations", [])
        dependencies = node.get("dependencies", [])
        
        for symbol in symbols:
            symbol_info = self.get_symbol_info(symbol, level)
            if not symbol_info:
                self.add_error(f"переменная '{symbol}' в составном присваивании не объявлена", 
                             scope_idx, node_idx)
            else:
                # Проверяем, не была ли переменная удалена
                if self.is_variable_deleted(symbol, level):
                    self.add_error(f"переменная '{symbol}' была удалена и требует переинициализации", 
                                 scope_idx, node_idx)
                elif symbol_info.get("key") == "const":
                    self.add_error(f"попытка модификации константы '{symbol}'", 
                                 scope_idx, node_idx)
        
        for dep in dependencies:
            if not self.find_symbol_in_scope(dep, level):
                self.add_error(f"используемая переменная '{dep}' не объявлена", 
                             scope_idx, node_idx)
            elif self.is_variable_deleted(dep, level):
                self.add_error(f"используемая переменная '{dep}' была удалена", 
                             scope_idx, node_idx)
    
    def validate_function_declaration(self, node: Dict, node_idx: int, scope_idx: int, 
                                  symbol_table: Dict, level: int):
        """Валидирует объявление функции"""
        func_name = node.get("function_name")
        parameters = node.get("parameters", [])
        
        for other_scope in self.all_scopes:
            if other_scope.get("level") <= level:
                for other_node in other_scope.get("graph", []):
                    if (other_node.get("node") == "function_declaration" and 
                        other_node.get("function_name") == func_name and 
                        other_node is not node):
                        self.add_error(f"функция '{func_name}' уже объявлена", 
                                     scope_idx, node_idx)
                        return
        
        param_names = set()
        for param in parameters:
            param_name = param.get("name")
            if param_name in param_names:
                self.add_error(f"дублирующийся параметр '{param_name}' в функции '{func_name}'", 
                             scope_idx, node_idx)
            param_names.add(param_name)
    
    def validate_function_call(self, node: Dict, node_idx: int, scope_idx: int, 
                             symbol_table: Dict, level: int):
        """Валидирует вызов функции"""
        func_name = node.get("function")
        arguments = node.get("arguments", [])
        
        # Проверяем, что функция существует или является встроенной
        if func_name not in self.functions and func_name not in self.builtin_functions:
            self.add_error(f"вызываемая функция '{func_name}' не объявлена", 
                         scope_idx, node_idx)
        elif func_name in self.functions:
            func_info = self.functions[func_name]
            func_params = func_info.get("parameters", [])
            
            if len(arguments) != len(func_params):
                self.add_error(f"функция '{func_name}' ожидает {len(func_params)} аргументов, передано {len(arguments)}", 
                             scope_idx, node_idx)
        
        # Проверяем аргументы
        for arg in arguments:
            if arg and arg.isalpha() and arg not in ["True", "False", "None"]:
                if not self.find_symbol_in_scope(arg, level):
                    self.add_error(f"аргумент '{arg}' не объявлен", 
                                 scope_idx, node_idx)
                elif self.is_variable_deleted(arg, level):
                    self.add_error(f"аргумент '{arg}' был удален", 
                                 scope_idx, node_idx)
    
    def validate_builtin_function_call(self, node: Dict, node_idx: int, scope_idx: int, 
                                  symbol_table: Dict, level: int):
        """Валидирует вызов встроенной функции"""
        func_name = node.get("function")
        arguments = node.get("arguments", [])
        dependencies = node.get("dependencies", [])
        
        if func_name not in self.builtin_functions:
            self.add_error(f"встроенная функция '{func_name}' не поддерживается", 
                        scope_idx, node_idx)
            return
        
        # ... остальная валидация ...
        
        # Проверяем зависимости (переменные в аргументах)
        for dep in dependencies:
            if not self.find_symbol_in_scope(dep, level):
                self.add_error(f"переменная '{dep}' в аргументе функции '{func_name}' не объявлена", 
                            scope_idx, node_idx)
            elif self.is_variable_deleted(dep, level):
                # Но нужно проверить, не было ли использование раньше удаления
                node_id = f"{scope_idx}.{node_idx}"
                current_timestamp = None
                for action in self.variable_history.get((level, dep), []):
                    if action.get("node_id") == node_id:
                        current_timestamp = action.get("timestamp")
                        break
                
                if current_timestamp is not None:
                    # Ищем удаление после этого использования
                    found_delete_after = False
                    for action in self.variable_history.get((level, dep), []):
                        if (action["action"] == "delete" and 
                            action["timestamp"] > current_timestamp):
                            found_delete_after = True
                            break
                    
                    if not found_delete_after:
                        self.add_error(f"переменная '{dep}' в аргументе функции '{func_name}' была удалена", 
                                    scope_idx, node_idx)
    
    def validate_print(self, node: Dict, node_idx: int, scope_idx: int, 
                      symbol_table: Dict, level: int):
        """Валидирует вызов функции print"""
        arguments = node.get("arguments", [])
        dependencies = node.get("dependencies", [])
        
        for dep in dependencies:
            if not self.find_symbol_in_scope(dep, level):
                self.add_error(f"переменная '{dep}' в аргументе print не объявлена", 
                            scope_idx, node_idx)
            elif self.is_variable_deleted(dep, level):
                self.add_error(f"переменная '{dep}' в аргументе print была удалена", 
                            scope_idx, node_idx)
        
        # Дополнительно проверяем аргументы для сложных выражений
        for arg in arguments:
            if (arg and 
                not arg.startswith('"') and not arg.endswith('"') and
                not arg.startswith("'") and not arg.endswith("'") and
                not arg.isdigit() and arg not in ["True", "False", "None"]):
                
                # Ищем переменные в сложных выражениях
                var_pattern = r'([a-zA-Z_][a-zA-Z0-9_]*)'
                vars_in_arg = re.findall(var_pattern, arg)
                for var in vars_in_arg:
                    if (var not in ["True", "False", "None"] and 
                        var not in self.builtin_functions and
                        not self.find_symbol_in_scope(var, level)):
                        self.add_error(f"переменная '{var}' в выражении print не объявлена", 
                                    scope_idx, node_idx)
                    elif self.is_variable_deleted(var, level):
                        self.add_error(f"переменная '{var}' в выражении print была удалена", 
                                    scope_idx, node_idx)
    
    def validate_len_call(self, node: Dict, node_idx: int, scope_idx: int, 
                         symbol_table: Dict, level: int):
        """Валидирует вызов len()"""
        arguments = node.get("arguments", [])
        
        if len(arguments) != 1:
            return
        
        arg = arguments[0]
        
        # len() принимает строки или массивы (пока только строки)
        if arg.startswith('"') and arg.endswith('"'):
            return
        
        if arg.startswith("'") and arg.endswith("'"):
            return
        
        # Для переменных нужно проверить тип
        if arg.isalpha():
            symbol_info = self.get_symbol_info(arg, level)
            if symbol_info:
                var_type = symbol_info.get("type")
                if var_type not in ["string", "list", "array"]:
                    self.add_error(f"функция len() ожидает строку, передана переменная типа '{var_type}'", 
                                 scope_idx, node_idx)
    
    def validate_int_call(self, node: Dict, node_idx: int, scope_idx: int, 
                         symbol_table: Dict, level: int):
        """Валидирует вызов int()"""
        arguments = node.get("arguments", [])
        
        if len(arguments) != 1:
            return
        
        arg = arguments[0]
        
        # int() принимает числа, строки с числами или bool
        if arg.isdigit() or arg in ["True", "False"]:
            return
        
        if arg.startswith('"') and arg.endswith('"'):
            str_value = arg[1:-1]
            if not str_value.lstrip('-').isdigit():
                self.add_error(f"функция int() не может преобразовать строку '{arg}' в число", 
                             scope_idx, node_idx)
        
        # Для переменных проверяем тип
        if arg.isalpha():
            symbol_info = self.get_symbol_info(arg, level)
            if symbol_info:
                var_type = symbol_info.get("type")
                if var_type not in ["int", "string", "bool"]:
                    self.add_error(f"функция int() ожидает int, string или bool, передана переменная типа '{var_type}'", 
                                 scope_idx, node_idx)
    
    def validate_str_call(self, node: Dict, node_idx: int, scope_idx: int, 
                         symbol_table: Dict, level: int):
        """Валидирует вызов str()"""
        arguments = node.get("arguments", [])
        
        if len(arguments) != 1:
            return
        
        # str() принимает любые значения
        # Проверяем только, что аргумент существует
        arg = arguments[0]
        if arg.isalpha() and not self.find_symbol_in_scope(arg, level):
            self.add_error(f"переменная '{arg}' в аргументе str() не объявлена", 
                         scope_idx, node_idx)
        elif arg.isalpha() and self.is_variable_deleted(arg, level):
            self.add_error(f"переменная '{arg}' в аргументе str() была удалена", 
                         scope_idx, node_idx)
    
    def validate_bool_call(self, node: Dict, node_idx: int, scope_idx: int, 
                          symbol_table: Dict, level: int):
        """Валидирует вызов bool()"""
        arguments = node.get("arguments", [])
        
        if len(arguments) != 1:
            return
        
        # bool() принимает любые значения
        # Проверяем только, что аргумент существует
        arg = arguments[0]
        if arg.isalpha() and not self.find_symbol_in_scope(arg, level):
            self.add_error(f"переменная '{arg}' в аргументе bool() не объявлена", 
                         scope_idx, node_idx)
        elif arg.isalpha() and self.is_variable_deleted(arg, level):
            self.add_error(f"переменная '{arg}' в аргументе bool() была удалена", 
                         scope_idx, node_idx)
    
    def validate_range_call(self, node: Dict, node_idx: int, scope_idx: int, 
                           symbol_table: Dict, level: int):
        """Валидирует вызов range()"""
        arguments = node.get("arguments", [])
        
        # range() принимает 1-3 аргумента, все должны быть int
        for i, arg in enumerate(arguments):
            if arg.isdigit():
                continue
            
            if arg.isalpha():
                symbol_info = self.get_symbol_info(arg, level)
                if symbol_info:
                    var_type = symbol_info.get("type")
                    if var_type != "int":
                        self.add_error(f"аргумент {i+1} функции range() должен быть int, передана переменная типа '{var_type}'", 
                                     scope_idx, node_idx)
                else:
                    self.add_error(f"переменная '{arg}' в аргументе range() не объявлена", 
                                 scope_idx, node_idx)
                
                # Проверяем, не удалена ли переменная
                if self.is_variable_deleted(arg, level):
                    self.add_error(f"переменная '{arg}' в аргументе range() была удалена", 
                                 scope_idx, node_idx)
    
    def validate_return(self, node: Dict, node_idx: int, scope_idx: int, 
                       symbol_table: Dict, level: int):
        """Валидирует оператор return"""
        dependencies = node.get("dependencies", [])
        
        for dep in dependencies:
            if not self.find_symbol_in_scope(dep, level):
                self.add_error(f"возвращаемая переменная '{dep}' не объявлена", 
                             scope_idx, node_idx)
            elif self.is_variable_deleted(dep, level):
                self.add_error(f"возвращаемая переменная '{dep}' была удалена", 
                             scope_idx, node_idx)
    
    def validate_loop_node(self, node: Dict, node_idx: int, scope_idx: int, 
                          symbol_table: Dict, level: int):
        """Валидирует узел цикла"""
        node_type = node.get("node")
        
        if node_type == "while_loop":
            condition = node.get("condition", {})
            if condition.get("type") == "COMPARISON":
                left = condition.get("left")
                right = condition.get("right")
                
                for var in [left, right]:
                    if var and var.isalpha() and var not in ["True", "False", "None"]:
                        if not self.find_symbol_in_scope(var, level):
                            self.add_error(f"переменная '{var}' в условии цикла не объявлена", 
                                         scope_idx, node_idx)
                        elif self.is_variable_deleted(var, level):
                            self.add_error(f"переменная '{var}' в условии цикла была удалена", 
                                         scope_idx, node_idx)
        
        elif node_type == "for_loop":
            loop_var = node.get("loop_variable")
            iterable = node.get("iterable", {})
            
            if loop_var not in symbol_table:
                self.add_error(f"переменная цикла '{loop_var}' не объявлена", 
                             scope_idx, node_idx)
            elif self.is_variable_deleted(loop_var, level):
                self.add_error(f"переменная цикла '{loop_var}' была удалена", 
                             scope_idx, node_idx)
            
            if iterable.get("type") == "RANGE_CALL":
                args = iterable.get("arguments", {})
                for arg_name, arg_value in args.items():
                    if arg_value and arg_value.isalpha() and arg_value not in ["True", "False", "None"]:
                        if not self.find_symbol_in_scope(arg_value, level):
                            self.add_error(f"аргумент range '{arg_value}' не объявлен", 
                                         scope_idx, node_idx)
                        elif self.is_variable_deleted(arg_value, level):
                            self.add_error(f"аргумент range '{arg_value}' был удален", 
                                         scope_idx, node_idx)
    
    def validate_function_return(self, scope: Dict, scope_idx: int):
        """Проверяет, что функция имеет return если нужно"""
        return_info = scope.get("return_info", {})
        return_type = scope.get("return_type", "None")
        
        if return_type != "None" and not return_info.get("has_return", False):
            func_content = ""
            for node_idx, node in enumerate(scope.get("graph", [])):
                if node.get("node") == "function_declaration":
                    func_content = node.get("content", "")
                    break
            
            if func_content:
                self.add_warning(f"функция возвращает '{return_type}' но не имеет оператора return", 
                               scope_idx, None)
    
    def validate_loops(self, scope: Dict, scope_idx: int):
        """Проверяет циклы на корректность"""
        graph = scope.get("graph", [])
        
        for node_idx, node in enumerate(graph):
            if node.get("node") in ["while_loop", "for_loop"]:
                body = node.get("body", [])
                if not body:
                    self.add_warning(f"тело цикла пустое", scope_idx, node_idx)
    
    def find_symbol_in_scope(self, symbol_name: str, current_level: int) -> bool:
        """Ищет символ в текущем или родительских scope'ах"""
        # Проверяем текущий scope
        if current_level in self.scope_symbols and symbol_name in self.scope_symbols[current_level]:
            return True
        
        # Находим текущий scope в all_scopes
        current_scope = None
        for scope in self.all_scopes:
            if scope.get("level") == current_level:
                current_scope = scope
                break
        
        if current_scope:
            # Проверяем родительский scope
            parent_level = current_scope.get("parent_scope")
            if parent_level is not None:
                return self.find_symbol_in_scope(symbol_name, parent_level)
        
        return False
    
    def get_symbol_info(self, symbol_name: str, current_level: int) -> Optional[Dict]:
        """Получает информацию о символе из текущего или родительских scope'ов"""
        # Сначала проверяем текущий scope
        if (current_level in self.scope_symbols and 
            symbol_name in self.scope_symbols[current_level]):
            return self.scope_symbols[current_level][symbol_name]
        
        # Если не нашли в текущем scope, ищем во всех scope'ах
        for level, symbols in self.scope_symbols.items():
            if symbol_name in symbols:
                return symbols[symbol_name]
        
        return None
    
    def get_report(self) -> Dict:
        """Возвращает отчет о проверке"""
        return {
            "is_valid": len(self.errors) == 0,
            "error_count": len(self.errors),
            "warning_count": len(self.warnings),
            "errors": self.errors,
            "warnings": self.warnings
        }



# Тестовый пример
if __name__ == "__main__":
    path = "/Users/phil/GitHub/phils_language/parsed_code.json"

    with open(path, "r") as file:
        data = json.load(file)
    
    validator = JSONValidator()
    result = validator.validate(data)
    
    print("Результат валидации:")
    print(f"Валидный: {result['is_valid']}")
    print(f"Ошибок: {result['error_count']}")
    print(f"Предупреждений: {result['warning_count']}")
    
    if result['errors']:
        print("\nОшибки:")
        for error in result['errors']:
            print(f"  - {error}")
    
    if result['warnings']:
        print("\nПредупреждения:")
        for warning in result['warnings']:
            print(f"  - {warning}")

  Объявление переменной 'a' с инициализацией '10'
    Валидация выражения: '10'
  Объявление переменной 'b' с инициализацией '100'
    Валидация выражения: '100'
  Объявление переменной 'c' с инициализацией '0'
    Валидация выражения: '0'
    >>> Валидация assignment: c = mul(a, b)
  Проверка переменной 'c': symbol_info=True
    Состояние переменной 'c': active
    Удалена ли: False
  Выражение в правой части: 'mul(a, b)'
    Валидация выражения: 'mul(a, b)'
      Обнаружен вызов функции: mul
  Объявление переменной 'x' с инициализацией 'dd'
    Валидация выражения: 'dd'
      Проверка переменной 'dd' в выражении
Результат валидации:
Валидный: False
Ошибок: 2
Предупреждений: 0

Ошибки:
  - Строка 'c = mul(a, b)': функция 'mul' не объявлена
  - Строка 'var x: int = dd': переменная 'dd' в выражении не объявлена
